In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

d:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\python\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; 

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding = 'bytes')
        return data[b'data'], data[b'labels']

# tensorflow.Dataset
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
            # 下面的代码是做了一个 filter 
#             for item, label in zip(data, labels):
#                 if label in [0, 1]:
#                     all_data.append(item)
#                     all_labels.append(label)
        # vstack是将数据纵向合并到一起，而 hstack 是将数据水平合并到一起
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0, 1, 2, 3, 4, 5] -> [5, 3, 2, 4, 0, 1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception('have no more examples')
        if end_indicator > self._num_examples:
            raise Exception('batch size is larger than all examples')
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]
train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(10)
# print(batch_data)
# print(batch_labels)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
x = tf.placeholder(tf.float32, [None, 3072])
# y 的 shape 是 [None], eg: [0, 5, 6, 3]
y = tf.placeholder(tf.int64, [None])
# 在中间层我们需要加入一个非线性的变化函数
hidden1 = tf.layers.dense(x, 100, activation = tf.nn.relu)
hidden2 = tf.layers.dense(hidden1, 100, activation = tf.nn.relu)
hidden3 = tf.layers.dense(hidden2, 50, activation = tf.nn.relu)
# dense函数就是实现全连接的封装的api
y_ = tf.layers.dense(hidden3, 10)

# mean square loss 平方差损失函数
# course: 1 + e^x
# api: e^x / sum(e^x)
# [[0.01, 0.9, ..., 0.03], []]
# p_y = tf.nn.softmax(y_)
# 5 -> [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
# y_one_hot = tf.one_hot(y, 10, dtype = tf.float32)
# tf.reduce_mean是求均值，tf.square是求平方
# loss = tf.reduce_mean(tf.square(y_one_hot - p_y))

# 下面是交叉熵损失函数
loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)
# y_ -> softmax
# y -> one_hot
# loss = ylog(y_)

# indices
predict = tf.argmax(y_, 1)
# [1, 0, 1, 1, 1, 0, 0, 0]
correct_prediction = tf.equal(predict, y)
# 因为下面是要求解均值，所以需要使用 float 类型的数据
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
# AdamOptimizer是梯度下降的一个变种，1e-3是初始化的学习率
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
# init()函数使用来执行初始化的
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict = {
                x: batch_data, 
                y: batch_labels})
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' \
                 %(i+1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step : %d, acc: %4.5f' %(i+1, test_acc))
                

[Train] Step: 500, loss: 1.76994, acc: 0.25000
[Train] Step: 1000, loss: 1.41340, acc: 0.55000
[Train] Step: 1500, loss: 1.30690, acc: 0.70000
[Train] Step: 2000, loss: 1.49396, acc: 0.45000
[Train] Step: 2500, loss: 0.96281, acc: 0.80000
[Train] Step: 3000, loss: 2.44306, acc: 0.25000
[Train] Step: 3500, loss: 1.18278, acc: 0.60000
[Train] Step: 4000, loss: 1.61112, acc: 0.45000
[Train] Step: 4500, loss: 1.35784, acc: 0.45000
[Train] Step: 5000, loss: 1.75846, acc: 0.50000
(10000, 3072)
(10000,)
[Test] Step : 5000, acc: 0.47700
[Train] Step: 5500, loss: 1.70868, acc: 0.45000
[Train] Step: 6000, loss: 1.50708, acc: 0.60000
[Train] Step: 6500, loss: 1.03210, acc: 0.60000
[Train] Step: 7000, loss: 0.91409, acc: 0.65000
[Train] Step: 7500, loss: 1.47632, acc: 0.55000
[Train] Step: 8000, loss: 1.42918, acc: 0.55000
[Train] Step: 8500, loss: 1.41062, acc: 0.55000
[Train] Step: 9000, loss: 1.03936, acc: 0.75000
[Train] Step: 9500, loss: 1.09273, acc: 0.60000
[Train] Step: 10000, loss: 0.9334